Code to extract data from wiki 

In [1]:
import requests
website_url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
import pandas as pd
soup = BeautifulSoup(website_url,'lxml')

Extracting table and storing it into dataframe

In [2]:
postcode=[]
borough=[]
neighbourhood=[]
for items in soup.find('table', class_='wikitable sortable').find_all('tr')[1::1]:
    data = items.find_all(['th','td'])
    try:
        postcode.append(data[0].text)        
        borough.append(data[1].text)
        neighbourhood.append(data[2].text)
    except IndexError:pass
#    print("{}|{}|{}".format(country,title,name))

postal_code_canada = pd.DataFrame(
    {'Postcode': postcode,
     'Borough': borough,
     'Neighbourhood': neighbourhood
    })


Aggregating data on the basis of "Neighbourhood"

In [3]:
postal_code_canada_1=pd.DataFrame(postal_code_canada.groupby(['Postcode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)])).reset_index()
postal_code_canada_1.replace('\n','', regex=True,inplace=True)
print(postal_code_canada_1.head(20))

   Postcode       Borough                                      Neighbourhood
0       M1A  Not assigned                                       Not assigned
1       M1B   Scarborough                                     Rouge, Malvern
2       M1C   Scarborough             Highland Creek, Rouge Hill, Port Union
3       M1E   Scarborough                  Guildwood, Morningside, West Hill
4       M1G   Scarborough                                             Woburn
5       M1H   Scarborough                                          Cedarbrae
6       M1J   Scarborough                                Scarborough Village
7       M1K   Scarborough        East Birchmount Park, Ionview, Kennedy Park
8       M1L   Scarborough                    Clairlea, Golden Mile, Oakridge
9       M1M   Scarborough    Cliffcrest, Cliffside, Scarborough Village West
10      M1N   Scarborough                        Birch Cliff, Cliffside West
11      M1P   Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...

Replacing "Neighbourhood" column data with "Borough" where "Neighbourhood"== "Not assigned\n"

In [4]:
for idx,row in postal_code_canada_1.iterrows():
    if  postal_code_canada_1.loc[idx,'Neighbourhood'] == 'Not assigned\n':
        postal_code_canada_1.loc[idx,'Neighbourhood'] = postal_code_canada_1.loc[idx,'Borough']

In [5]:
postal_code_canada_1.loc[postal_code_canada_1['Postcode'] == 'M7A']

,Postcode,Borough,Neighbourhood
120,M7A,Queen's Park,Not assigned


In [6]:
postal_code_canada_1.shape


(180, 3)

In [7]:
import pandas as pd
df_n=pd.read_csv('http://cocl.us/Geospatial_data')
#print(data)
merged_data=pd.merge(postal_code_canada_1, df_n, left_on='Postcode', right_on='Postal Code')
final_merged_data=merged_data[['Postcode','Borough','Neighbourhood','Latitude','Longitude']]
final_merged_data.head(20)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
